## Create a  PowerOps Client

A power ops client is a layer on top the regular CogniteClient that contains domain specific interface against CDF and the powerops data models. See quickstart guide for setting up. 

In [1]:
from cognite.powerops import PowerOpsClient
from cognite.client import CogniteClient
import os
os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"  # settings files are not in current dir

power = PowerOpsClient.from_settings()
# The regular cognite client is available under the .cdf attribute.
client: CogniteClient = power.cdf

Note it is important that the `CogShop` version have been set to trigger SHOP runs.

## Fetching SHOP Runs

### Listing SHOP runs

In [2]:
shop_runs = power.shop.list(limit=5, watercourse="Fornebu")

In [3]:
shop_runs

,external_id,watercourse,start,end,case_file_external_id,shop_files_external_ids,shop_version
0,POWEROPS_SHOP_RUN_2023-09-09T09:56:11.284030Z_...,Fornebu,2021-05-21 22:00:00+00:00,2021-05-30 22:00:00+00:00,cog_shop_preprocessor/cogShop/Case_2023-09-09T...,[{'external_id': 'SHOP_Fornebu_water_value_cut...,14.4.3.0
1,POWEROPS_SHOP_RUN_2023-10-06T09:55:48.507978Z_...,Fornebu,2021-05-21 22:00:00+00:00,2021-05-30 22:00:00+00:00,cog_shop_preprocessor/cogShop/Case_2023-10-06T...,[{'external_id': 'SHOP_Fornebu_water_value_cut...,14.4.3.0
2,POWEROPS_SHOP_RUN_2023-10-13T09:55:56.341479Z_...,Fornebu,2021-05-21 22:00:00+00:00,2021-05-30 22:00:00+00:00,cog_shop_preprocessor/cogShop/Case_2023-10-13T...,[{'external_id': 'SHOP_Fornebu_water_value_cut...,14.4.3.0
3,POWEROPS_SHOP_RUN_2023-09-26T09:56:10.631069Z_...,Fornebu,2021-05-21 22:00:00+00:00,2021-05-30 22:00:00+00:00,cog_shop_preprocessor/cogShop/Case_2023-09-26T...,[{'external_id': 'SHOP_Fornebu_water_value_cut...,14.4.3.0
4,POWEROPS_SHOP_RUN_copied_1695903354396214957_P...,Fornebu,2021-05-21 22:00:00+00:00,2021-05-30 22:00:00+00:00,cog_shop_preprocessor/cogShop/Case_2023-09-27T...,[{'external_id': 'SHOP_Fornebu_water_value_cut...,14.4.3.0


### Listing with filters 
* Filtering on the start time and end time. The expected format is `YYYY-MM-DD` as seen in this example: 
* The there is also the option to filter on multiple watercourses by passing a list

In [4]:
power.shop.list(limit=5, start_after="2022-01-01", watercourse=["Fornebu"])

,external_id,watercourse,start,end,case_file_external_id,shop_files_external_ids,shop_version
0,POWEROPS_SHOP_RUN_2023-09-08T14:42:09.163927Z_...,Fornebu,2022-08-03 22:00:00+00:00,2022-08-14 22:00:00+00:00,cog_shop_preprocessor/cogShop/Case_2023-09-08T...,[{'external_id': 'SHOP_Fornebu_water_value_cut...,14.4.3.0
1,POWEROPS_SHOP_RUN_2023-09-08T15:23:45.100354Z_...,Fornebu,2022-08-03 22:00:00+00:00,2022-08-14 22:00:00+00:00,cog_shop_preprocessor/cogShop/Case_2023-09-08T...,[{'external_id': 'SHOP_Fornebu_water_value_cut...,14.4.3.0
2,POWEROPS_SHOP_RUN_2023-09-08T14:42:04.889488Z_...,Fornebu,2022-08-03 22:00:00+00:00,2022-08-14 22:00:00+00:00,cog_shop_preprocessor/cogShop/Case_2023-09-08T...,[{'external_id': 'SHOP_Fornebu_water_value_cut...,14.4.3.0
3,POWEROPS_SHOP_RUN_2023-09-08T15:23:45.101388Z_...,Fornebu,2022-08-03 22:00:00+00:00,2022-08-14 22:00:00+00:00,cog_shop_preprocessor/cogShop/Case_2023-09-08T...,[{'external_id': 'SHOP_Fornebu_water_value_cut...,14.4.3.0
4,POWEROPS_SHOP_RUN_2023-09-08T14:42:09.164453Z_...,Fornebu,2022-08-03 22:00:00+00:00,2022-08-14 22:00:00+00:00,cog_shop_preprocessor/cogShop/Case_2023-09-08T...,[{'external_id': 'SHOP_Fornebu_water_value_cut...,14.4.3.0


### Retrieving SHOP run(s)

In [5]:
run = power.shop.retrieve(shop_runs[0].external_id)

In [6]:
run

,Value
external_id,POWEROPS_SHOP_RUN_2023-09-09T09:56:11.284030Z_...
watercourse,Fornebu
start,2021-05-21 22:00:00+00:00
end,2021-05-30 22:00:00+00:00
case_file_external_id,cog_shop_preprocessor/cogShop/Case_2023-09-09T...
shop_files_external_ids,[{'external_id': 'SHOP_Fornebu_water_value_cut...
shop_version,14.4.3.0


### Retrieving latest SHOP run


In [7]:
power.shop.retrieve_latest(watercourse="Fornebu", latest_by="created_time")

,Value
external_id,SHOP_RUN_2023-10-28T09:44:52.746485Z_b17160
watercourse,Fornebu
start,2023-10-28 09:44:52.746000+00:00
end,None
case_file_external_id,cog_shop_manual_2023-10-28T09:44:52.746485Z_c5...
shop_files_external_ids,[]
shop_version,CogShop2-20230710T163856Z


In [8]:
power.shop.retrieve_latest(watercourse="Fornebu", latest_by="created_time", end_before="2023-09-01")

,Value
external_id,POWEROPS_SHOP_RUN_2023-10-27T13:29:00.888780Z_...
watercourse,Fornebu
start,2022-09-21 22:00:00+00:00
end,2022-10-02 22:00:00+00:00
case_file_external_id,cog_shop_preprocessor/cogShop/Case_2023-10-27T...
shop_files_external_ids,[{'external_id': 'SHOP_Fornebu_water_value_cut...
shop_version,14.4.3.0


### Inspecting SHOP Run Input Files

The `SHOPRun` object gives access to the input files and produced outputs (in the case of a successful run).

In [9]:
from IPython.display import Markdown, display

In [10]:
# A small helper to illustrate
def display_yaml(yaml_str):
    display(Markdown(f"```yaml\n{yaml_str}```"))

In [11]:
run.check_status()

<SHOPRunStatus.SUCCESS: 'success'>

In [12]:
display_yaml(run.get_case_file()[:500])

```yaml
commands:
- set time_delay_unit MINUTE
- set ramping /on
- set stop_cost_from_start_cost /on
- set bypass_loss /on
- set mipgap 0.001000
- set timelimit 600.000
- set reserve_ramping_cost 1
- set fcr_n_equality /on
- set reserve_slack_cost 1
- set reserve_min_capacity 0.02
- set dyn_seg /on
- set dyn_juncloss /on
- penalty flag /on /plant /schedule
- start sim 3
- set code /inc
- start sim 4
connections:
- connection_type: connection_bypass
  from: Hagen
  from_type: reservoir
  to: b_Hagen_Stra```

In [13]:
for content in run.get_shop_files():
    display_yaml(content[:50])

```yaml
NAMELIST

# {reservoir name in SHOP} {correspond```

```yaml
 SHOP_WATER_VALUES
         500          15
    ```

### Inspecting SHOP Run Output Files

In [14]:
for log_file, content in run.get_log_files():
    display(Markdown(f"**{log_file}**"))
    display_yaml(content[:500])

**POWEROPS_SHOP_post-run-4e089cad-5c9e-46e2-9c87-32fd6d9f1f81.yaml**

```yaml
time:
  starttime: 2021-05-21 22:00:00
  endtime: 2021-05-30 22:00:00
  timeunit: minute
  timeresolution:
    2021-05-21 22:00:00: 60
    2021-05-24 22:00:00: 240
model:
  objective:
    average_objective:
      solver_status: Optimal solution is available
      grand_total: -18798762081.974
      total: -18798857928.877
      sum_penalties: 95846.902692765
      minor_penalties: 95846.902692765
      rsv_end_value: -18797354080.554
      rsv_end_value_relative: -187919009.72324
      market_sa```

**POWEROPS_SHOP_shop_messages-4e089cad-5c9e-46e2-9c87-32fd6d9f1f81.log**

```yaml
DIAGNOSIS WARNING: 3250
Reservoir Dalbysvatn: Only max tactical cost or limit is given.
DIAGNOSIS WARNING: 3205
Reservoir Dalbysvatn: 0 errors 1 warnings.
--------------*----------------*------------
DIAGNOSIS WARNING: 3250
Reservoir Danielsen: Only max tactical cost or limit is given.
DIAGNOSIS WARNING: 3205
Reservoir Danielsen: 0 errors 1 warnings.
--------------*----------------*------------
DIAGNOSIS WARNING: 3250
Reservoir Sirefelt: Only max tactical cost or limit is given.
DIAGNOSIS WARNIN```

**POWEROPS_SHOP_cplex-4e089cad-5c9e-46e2-9c87-32fd6d9f1f81.log**

```yaml
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Parallel                                1
CPXPARAM_Simplex_Limits_Iterations               10000000
CPXPARAM_MIP_Interval                            100
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Tolerances_MIPGap                   0.001
Tried aggregator 3 times.
MIP Presolve eliminated 1727 rows and 1636 columns.
MIP Presolve modified 1938 coefficients.
Aggregator did 5699 substitutions.
Reduced MIP has 16926 ```

## Trigger SHOP Run

Here we are reusing an old case file for demo purposes

In [15]:
import time
from cognite.powerops.client.data_classes import SHOPRunStatus
from cognite.powerops.client.shop.shop_case import SHOPCase

In [16]:
new_case_data = run.get_case_file()
new_case = SHOPCase(new_case_data, watercourse="Fornebu")

In [17]:
MAX_WAIT_IN_SECONDS = 120

In [18]:
t0 = time.time()

new_run = power.shop.trigger_case(new_case)
# Waiting for SHOP run to finish
while (time.time() - t0) < MAX_WAIT_IN_SECONDS:
    if new_run.check_status() is SHOPRunStatus.SUCCESS:
        print(f"RUN is done in {time.time() -t0:.2f} seconds")
        break
    time.sleep(1)
    print(".", end="")

........................RUN is done in 26.84 seconds


In [19]:
for log_file, content in new_run.get_log_files():
    display(Markdown(f"**{log_file}**"))
    display_yaml(content[:500])

**cog_shop_manual_2023-10-28T09:45:35.131763Z_fed409**

```yaml
commands:
- set time_delay_unit MINUTE
- set ramping /on
- set stop_cost_from_start_cost /on
- set bypass_loss /on
- set mipgap 0.001000
- set timelimit 600.000
- set reserve_ramping_cost 1
- set fcr_n_equality /on
- set reserve_slack_cost 1
- set reserve_min_capacity 0.02
- set dyn_seg /on
- set dyn_juncloss /on
- penalty flag /on /plant /schedule
- start sim 3
- set code /inc
- start sim 4
connections:
- connection_type: connection_bypass
  from: Hagen
  from_type: reservoir
  to: b_Hagen_Stra```

**POWEROPS_SHOP_shop_messages-fb235d06-f76d-4e80-92e2-3dd4f9371fd4.log**

```yaml
DIAGNOSIS WARNING: 3250
Reservoir Dalbysvatn: Only max tactical cost or limit is given.
DIAGNOSIS WARNING: 3205
Reservoir Dalbysvatn: 0 errors 1 warnings.
--------------*----------------*------------
DIAGNOSIS WARNING: 3250
Reservoir Danielsen: Only max tactical cost or limit is given.
DIAGNOSIS WARNING: 3205
Reservoir Danielsen: 0 errors 1 warnings.
--------------*----------------*------------
DIAGNOSIS WARNING: 3250
Reservoir Sirefelt: Only max tactical cost or limit is given.
DIAGNOSIS WARNIN```

**POWEROPS_SHOP_cplex-fb235d06-f76d-4e80-92e2-3dd4f9371fd4.log**

```yaml
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Parallel                                1
CPXPARAM_Simplex_Limits_Iterations               10000000
CPXPARAM_MIP_Interval                            100
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Tolerances_MIPGap                   0.001
Tried aggregator 3 times.
MIP Presolve eliminated 1726 rows and 1636 columns.
MIP Presolve modified 1938 coefficients.
Aggregator did 5698 substitutions.
Reduced MIP has 16428 ```

**POWEROPS_SHOP_post-run-fb235d06-f76d-4e80-92e2-3dd4f9371fd4.yaml**

```yaml
time:
  starttime: 2021-05-21 22:00:00
  endtime: 2021-05-30 22:00:00
  timeunit: minute
  timeresolution:
    2021-05-21 22:00:00: 60
    2021-05-24 22:00:00: 240
model:
  objective:
    average_objective:
      solver_status: Optimal solution is available
      grand_total: -41247784.371522
      total: -41306371.070504
      sum_penalties: 58586.698981099
      minor_penalties: 58586.698981099
      rsv_end_value: -39763969.745769
      rsv_end_value_relative: -17760966.59443
      rsv_spill_```

In [20]:
new_run

,Value
external_id,SHOP_RUN_2023-10-28T09:45:35.131763Z_63458b
watercourse,Fornebu
start,2023-10-28 09:45:35.131763+00:00
end,None
case_file_external_id,cog_shop_manual_2023-10-28T09:45:35.131763Z_fe...
shop_files_external_ids,[]
shop_version,CogShop2-20230710T163856Z


For more information about the SHOP client, see the `client` section in the reference documentation.

End of notebook.